# Prototype Visualization of Structural Variation Output

set up bokeh for notebook drawing, and import gene_viz

In [1]:
from bokeh.plotting import output_notebook, show
output_notebook(hide_banner=True)

import gene_viz
from gene_viz.features import Transcript, Exon, CDS

## Use PyEnsembl as the data source

In [2]:
import os
import pyensembl

# set database cache directory for pyensembl
os.environ['PYENSEMBL_CACHE_DIR'] = os.path.abspath("./")

# download and unzip gencode basic annotations for hg38
#!wget ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_27/gencode.v27.basic.annotation.gtf.gz
#!gunzip gencode.v27.basic.annotation.gtf.gz
    
genome = pyensembl.Genome(
    reference_name="GRCh38",
    annotation_name="Gencode27",
    gtf_path_or_url="gencode.v27.basic.annotation.gtf.gz"
)
genome.index()

## Create a method to translate pyensembl objects into gene_viz features 

In [3]:
def transcripts_from_pyensembl(contig, start, end):
    """
    Get annotations from pyensembl and translate them into gene_viz features
    """
    transcripts = []

    for gene in genome.genes_at_locus(contig, start, end):
        for t in gene.transcripts:
            tr = Transcript(t.transcript_name, t.gene_id, t.contig, t.start, t.end, t.strand)
                
            for e in t.exons:
                tr.add_exon(Exon(e.id, e.contig, e.start, e.end))
        
            try:
                for c in t.coding_sequence_position_ranges:
                    tr.add_cds(CDS(t.contig, c[0], c[1]))
            except ValueError:
                pass
        
            transcripts.append(tr)
    
    return transcripts

## Create a gene_viz GenePlot

In [4]:
def make_plot(transcripts):
    """
    Create a gene_plot object from a list of transcript feature objects
    """
    start = min([t.start for t in transcripts])
    end = max([t.end for t in transcripts])
    contig = next((t.contig for t in transcripts), "unlabelled")
    
    gene_plot = gene_viz.GenePlot(
        dict(
            row_height=15,
            pack=True,
            show_labels=True,
            label_horiz_position="center",
            label_vert_position="above",
            label_justify="center",
            label_offset=(0, -4),
            label_font_size="6pt",
            intron_width_percent=0.01,
        )
    )
        
    gene_plot.x_range = (start, end)
    gene_plot.figure.xaxis.axis_label = contig
    gene_plot.transcripts = transcripts
    gene_plot.update()
    return gene_plot

## Query pyensembl and visualize the results

In [5]:
fig = make_plot(transcripts_from_pyensembl("chr22", 42124000, 42148000))
show(fig.figure)

# Do the same using GFFUtils as the data source

In [6]:
import gffutils
import sqlite3

## Set up a GFFUtils database using the same annotation data

In [7]:
# note:
# database creation using gffutils is slower than pyensembl on the same dataset
try:
    gffutils.create_db(
        "gencode.v27.basic.annotation.gtf.gz",
        "gencode.v27.basic.sqlite",
        keep_order=True,
        disable_infer_genes=True, disable_infer_transcripts=True)
except (sqlite3.OperationalError, AttributeError):
    pass

In [8]:
# connect
db = gffutils.FeatureDB("gencode.v27.basic.sqlite")
gffutils.constants.always_return_list = False

# Create a function to translate GFFUtils queries into gene_viz features

In [9]:
# translate into features for gene_viz
def transcripts_from_gffutils(contig, start, end):
    transcripts = []

    for gene in db.features_of_type("gene", limit=(contig, start, end)):
        trs = db.children(gene.id, featuretype="transcript")
        for t in trs:
            transcript = Transcript(t.attributes["transcript_name"], gene.attributes["gene_name"], t.seqid, t.start, t.end, t.strand)
        
            es = db.children(t.id, featuretype="exon")
            for e in es:
                transcript.add_exon(Exon(e.attributes["exon_id"], e.seqid, e.start, e.end))
            
            cs = db.children(t.id, featuretype="CDS")
            for c in cs:
                transcript.add_cds(CDS(c.seqid, c.start, c.end))
        
            transcripts.append(transcript)
    
    return transcripts

## Perform a query and visualize the results

In [10]:
fig = make_plot(transcripts_from_gffutils("chr22", 42124000, 42148000))
show(fig.figure)